In [1]:
# ALWAYS RUN THIS CELL
# widen jupyter notebook window
from IPython.display import display, HTML
display(HTML("<style>.container {width:95% !important; }</style>"))

In [2]:
import os
from pathlib import Path

import paramiko
from scp import SCPClient

import getpass
import gc
import time

In [3]:
dir_github = '/media/rich/Home_Linux_partition/github_repos/'
import sys
sys.path.append(dir_github)

%load_ext autoreload
%autoreload 2
from basic_neural_processing_modules import server

In [4]:
# paramiko.server.log_to_file("paramiko.log")

----
# ==== Optional ====
### transfer raw data from local machine to `MICROSCOPE`

In [19]:
## Define directories for where the local data is and the remote directory where the contents of that folder should be copied to (MICROSCOPE)
dir_data_local = '/media/rich/bigSSD/tmp_data/'
dir_data_MICROSCOPE = '/n/files/Neurobio/MICROSCOPE/Rich/data/test'  ## contents of dir_data_local will be dumped into dir_data_MICROSCOPE

print(f'contents of    {dir_data_local}    will be copied to    {dir_data_MICROSCOPE}')

contents of    /media/rich/bigSSD/tmp_data/    will be copied to    /n/files/Neurobio/MICROSCOPE/Rich/data/test


In [20]:
remote_host_transfer = "transfer.rc.hms.harvard.edu"
remote_host_compute = "o2.hms.harvard.edu"
username = input('Username: ')
pw = server.pw_encode(getpass.getpass(prompt='Password: '))

Username: rh183
Password: ········


## Prepare `sftp` and `ssh` objects

In [ ]:
sftp = server.sftp_interface(hostname="transfer.rc.hms.harvard.edu", port=22)
sftp.connect(username=username, password=server.pw_decode(pw))

In [30]:
# sftp.mkdir_safe(dir_data_remote)
sftp.put_dir(dir_data_local, dir_data_MICROSCOPE)

uploading /media/rich/bigSSD/tmp_data/day0_00001_00001.tif   to   /n/files/Neurobio/MICROSCOPE/Rich/data/test/day0_00001_00001.tif
uploading /media/rich/bigSSD/tmp_data/day0_00001_00002.tif   to   /n/files/Neurobio/MICROSCOPE/Rich/data/test/day0_00001_00002.tif


------------
# ==== Start ====

## Prepare username + password

In [5]:
remote_host_compute = "o2.hms.harvard.edu"
remote_host_transfer = "transfer.rc.hms.harvard.edu"
username = input('O2 Username: ')

use_localSshKey = True

pw = server.pw_encode(getpass.getpass(prompt='Password: ')) if use_localSshKey==False else None

path_sshKey = '/home/rich/.ssh/id_rsa' if use_localSshKey else None

O2 Username: rh183


## Prepare `sftp` and `ssh` objects

In [6]:
## initialize ssh_compute
ssh_t = server.ssh_interface(
    nbytes_toReceive=20000,
    recv_timeout=1,
    verbose=True,
)
ssh_t.o2_connect(
    hostname=remote_host_transfer,
    username=username,
    password=server.pw_decode(pw),
    key_filename=path_sshKey,
    look_for_keys=False,
    passcode_method=1,
    verbose=0,
    skip_passcode=False,    
)

In [7]:
sftp = server.sftp_interface(ssh_client=ssh_t.client)

In [8]:
## initialize ssh_compute
ssh_c = server.ssh_interface(
    nbytes_toReceive=20000,
    recv_timeout=1,
    verbose=True,
)
ssh_c.o2_connect(
    hostname=remote_host_compute,
    username=username,
    password=server.pw_decode(pw),
    key_filename=path_sshKey,
    look_for_keys=False,
    passcode_method=1,
    verbose=0,
    skip_passcode=False,    
)

## Pull/update repo

In [10]:
ssh_c.send('cd /n/data1/hms/neurobio/sabatini/rich/github_repos/NBAP')
ssh_c.send('git pull')

ssh_c.expect(str_success=f'[{username}');

remote: Enumerating objects: 21, done.
remote: Counting objects: 100% (21/21), done.
remote: Compressing objects: 100% (1/1), done.
remote: Total 11 (delta 10), reused 11 (delta 10), pack-reused 0
Unpacking objects: 100% (11/11), done.
From https://github.com/RichieHakim/NBAP
   be9b69a..38f9080  main       -> origin/main
Updating be9b69a..38f9080
Fast-forward
 .../clean_traces/clean_traces.py                   |   3 +-
 .../temporal_alignment-checkpoint.ipynb            | 158 +-----
 .../temporal_alignment/dispatcher.py               |  14 -
 .../temporal_alignment/master_controller.ipynb     | 603 ++++++++++++++++-----
 .../temporal_alignment/temporal_alignment.ipynb    | 233 --------
 .../temporal_alignment/temporal_alignment.py       |   2 +-
 6 files changed, 461 insertions(+), 552 deletions(-)
(base) [rh183@login05 NBAP]$ 


## Activate environment

In [11]:
name_env = '/n/data1/hms/neurobio/sabatini/rich/virtual_envs/fr_venv'

ssh_c.send_receive('')
ssh_c.send(f'conda activate {name_env}')
ssh_c.expect(str_success=f'[{username}', total_timeout=2);
ssh_c.send_receive('');

virtual_envs/fr_venvta1/hms/neurobio/sabatini/rich/ 

 /n/data1/hms/neurobio/sabatini/rich/virtual_envs/fr_venv) [rh183@login05 NBAP]$ 


## Prepare directories

In [12]:
import numpy as np

mouse_name = 'AEG22'
date = '2022_05_16'

name_slurm = mouse_name + '_' + date[-2:]

dir_s2p = (Path('/n/data1/hms/neurobio/sabatini/rich/analysis/suite2p_output/') / mouse_name / date / 'jobNum_0' / 'suite2p' / 'plane0').as_posix()
dir_ws = (Path('/n/data1/hms/neurobio/sabatini/rich/data/2pRAM/') / mouse_name / date / 'wavesurfer').as_posix()
dir_camera = (Path('/n/data1/hms/neurobio/sabatini/rich/data/2pRAM') / mouse_name / date / 'camera').as_posix()
dir_nwb = (Path('/n/data1/hms/neurobio/sabatini/rich/analysis/faceRhythm') / mouse_name / date / 'jobNum_0' / 'batchRun' / 'data').as_posix()

In [13]:
path_ws = sftp.search_recursive(
    path=dir_ws,
    search_pattern_re='h5'
)[-1]

/n/data1/hms/neurobio/sabatini/rich/data/2pRAM/AEG22/2022_05_16/wavesurfer/AEG22_2022-05-16_0001.h5


In [14]:
path_cameraCSV = np.sort(np.array(sftp.search_recursive(
    path=dir_camera,
    search_pattern_re='csv',
    verbose=False
)))[-1]
path_cameraCSV

'/n/data1/hms/neurobio/sabatini/rich/data/2pRAM/AEG22/2022_05_16/camera/20220516AEG22_csv32022-05-16T18_01_41.csv'

In [15]:
path_nwb = sftp.search_recursive(
    path=dir_nwb,
    search_pattern_re='nwb'
)[-1]

/n/data1/hms/neurobio/sabatini/rich/analysis/faceRhythm/AEG22/2022_05_16/jobNum_0/batchRun/data/session_batch.nwb


In [16]:
dir_saveOutputs = (Path('/n/data1/hms/neurobio/sabatini/rich/analysis/temporal_alignment') / mouse_name / date).as_posix()

name_job = 'jobNum_'


# path_dispatcher_local = Path(r'C:\Users\scanimage\github_repos\s2p_on_o2').resolve() / 'dispatcher.py'  ## path to the dispatcher.py file on local computer
path_dispatcher_local = Path(r'/media/rich/Home_Linux_partition/github_repos/NBAP/pipeline_2pRAM_faceRhythm/temporal_alignment/').resolve() / 'dispatcher.py'  ## path to the dispatcher.py file on local computer
path_dispatcher_remote = (Path(dir_saveOutputs) / 'dispatcher.py').as_posix()  ## path to where you want the dispatcher.py file to be copied onto the server
path_script_remote = Path(r'/n/data1/hms/neurobio/sabatini/rich/github_repos/NBAP/pipeline_2pRAM_faceRhythm/temporal_alignment/temporal_alignment_justTemplates.py').as_posix()  ## path to a copy of the remote_run_s2p.py file within the s2p_on_o2 repo on the server


## Prepare commands

In [17]:
## Expectation for dispatch args: 
# path_selfScript = args[0] = path_dispatcher_remote
# dir_save = args[1] = dir_S2pOutput_remote
# path_script = args[2] = path_s2pScript_remote
# name_job = args[3] = name_job
# dir_fastDisk = args[4] = dir_fastDisk_remote
# name_slurm = args[5] = name_slurm
# dir_data = args[6] = dir_data_remote


prompt_snip = f'[{username}'

commands = {
    'make_dir': f"mkdir -p {dir_saveOutputs}",
    'dispatch_s2p': f"python {str(path_dispatcher_remote)} {path_script_remote} {dir_saveOutputs} {dir_s2p} {path_ws} {path_cameraCSV} {path_nwb} {name_job} {name_slurm} {name_env}"  ## dispatcher expecting these args as inputs
}
display(commands)

{'make_dir': 'mkdir -p /n/data1/hms/neurobio/sabatini/rich/analysis/temporal_alignment/AEG22/2022_05_16',
 'dispatch_s2p': 'python /n/data1/hms/neurobio/sabatini/rich/analysis/temporal_alignment/AEG22/2022_05_16/dispatcher.py /n/data1/hms/neurobio/sabatini/rich/github_repos/NBAP/pipeline_2pRAM_faceRhythm/temporal_alignment/temporal_alignment_justTemplates.py /n/data1/hms/neurobio/sabatini/rich/analysis/temporal_alignment/AEG22/2022_05_16 /n/data1/hms/neurobio/sabatini/rich/analysis/suite2p_output/AEG22/2022_05_16/jobNum_0/suite2p/plane0 /n/data1/hms/neurobio/sabatini/rich/data/2pRAM/AEG22/2022_05_16/wavesurfer/AEG22_2022-05-16_0001.h5 /n/data1/hms/neurobio/sabatini/rich/data/2pRAM/AEG22/2022_05_16/camera/20220516AEG22_csv32022-05-16T18_01_41.csv /n/data1/hms/neurobio/sabatini/rich/analysis/faceRhythm/AEG22/2022_05_16/jobNum_0/batchRun/data/session_batch.nwb jobNum_ AEG22_16 /n/data1/hms/neurobio/sabatini/rich/virtual_envs/fr_venv'}

## Upload `dispatcher.py` file

In [18]:
sftp.mkdir_p(Path(path_dispatcher_remote).parent.as_posix())
sftp.sftp.put(str(path_dispatcher_local), str(path_dispatcher_remote));

## Copy data from `MICROSCOPE` to `data1`

## Dispatch `remote_run_s2p.py`

In [19]:
ssh_c.send(commands['dispatch_s2p']);

ssh_c.expect(str_success=prompt_snip);

/data1/hmAM_faceRhythm/temporal_alignment/temporal_alignment_justTemplates.py /n 
virtual_envs/fr_venvch.nwb jobNum_ AEG22_16 /n/data1/hms/neurobio/sabatini/rich/ 

dispatcher environment: /n/data1/hms/neurobio/sabatini/rich/virtual_envs/fr_venv

/n/data1/hms/neurobio/sabatini/rich/analysis/temporal_alignment/AEG22/2022_05_16/dispatcher.py /n/data1/hms/neurobio/sabatini/rich/github_repos/NBAP/pipeline_2pRAM_faceRhythm/temporal_alignment/temporal_alignment_justTemplates.py /n/data1/hms/neurobio/sabatini/rich/analysis/temporal_alignment/AEG22/2022_05_16 /n/data1/hms/neurobio/sabatini/rich/analysis/suite2p_output/AEG22/2022_05_16/jobNum_0/suite2p/plane0 /n/data1/hms/neurobio/sabatini/rich/data/2pRAM/AEG22/2022_05_16/wavesurfer/AEG22_2022-05-16_0001.h5 /n/data1/hms/neurobio/sabatini/rich/data/2pRAM/AEG22/2022_05_16/camera/20220516AEG22_csv32022-05-16T18_01_41.csv /n/data1/hms/neurobio/sabatini/rich/analysis/faceRhythm/AEG22/2022_05_16/jobNum_0/batchRun/data/session_batch.nwb jobNum_ AEG22_

### check on job

In [81]:
ssh_c.send(cmd=f'squeue -u {username} --Format=jobid,name,partition,state,timeused,starttime,timelimit,reasonlist,minmemory,mincpus,gres');
ssh_c.expect(str_success=f"[{username}", total_timeout=3);

r_venvch.nwb jobNum_ AEG22_16 /n/data1/hms/neurobio/sabatini/rich/virtual_envs/f 

squeue -u rh183 --Format=jobid,name,partition,state,timeused,starttime,timelimit,reasonlist,minmemory,mincpus,gres
dispatcher environment: /n/data1/hms/neurobio/sabatini/rich/virtual_envs/fr_venv

/n/data1/hms/neurobio/sabatini/rich/analysis/temporal_alignment/AEG22/2022_05_16/dispatcher.py /n/data1/hms/neurobio/sabatini/rich/github_repos/NBAP/pipeline_2pRAM_faceRhythm/temporal_alignment/temporal_alignment.py /n/data1/hms/neurobio/sabatini/rich/analysis/temporal_alignment/AEG22/2022_05_16 /n/data1/hms/neurobio/sabatini/rich/analysis/suite2p_output/AEG22/2022_05_16/jobNum_0/suite2p/plane0 /n/data1/hms/neurobio/sabatini/rich/data/2pRAM/AEG22/2022_05_16/wavesurfer/AEG22_2022-05-16_0001.h5 /n/data1/hms/neurobio/sabatini/rich/data/2pRAM/AEG22/2022_05_16/camera/20220516AEG22_csv32022-05-16T18_01_41.csv /n/data1/hms/neurobio/sabatini/rich/analysis/faceRhythm/AEG22/2022_05_16/jobNum_0/batchRun/data/session_batch

In [340]:
## I think this code block caused an out of memory error on a few of my jobs... not sure why

# ssh_c.send(cmd=f"cat {(Path(dir_S2pOutput_remote) / (name_job+'0') / 'print*').as_posix()}")
# ssh_c.expect(str_success=f"[{username}", total_timeout=1);

In [27]:
ssh_c.send(cmd=f"cat {(Path(dir_S2pOutput_remote) / (name_job+'0') / 'print*').as_posix()}")
ssh_c.expect(str_success=f"RUN COMPLETE", recv_timeout=0.3, total_timeout=60*60*10, sleep_time=0.1, verbose=False);

print(f'RUN COMPLETE!!!     {time.ctime()}')

RUN COMPLETE!!!     Tue May 17 14:52:20 2022


## Clean up

In [ ]:
## delete password
del pw
gc.collect()

## close sftp
if sftp: sftp.close()
if transport: transport.close()
del sftp
del transport
    
## close ssh
ssh_t.close()
del ssh_t
ssh_c.close()
del ssh_c
gc.collect()

----
# ==== Optional ====
### transfer s2p outputs back from `data1` to `MICROSCOPE`

In [44]:
def transfer(dir_from, dir_to, sftp, ssh_t, str_expect='[rh183', verbose=1, timeout_cp=10):
    """
    Makes a directory
    Transfers
    Checks to make sure the transfer worked
    """
    ssh_t.receive(timeout=0.1, verbose=False, throw_error=False) ## clean any lingering receives
    
    sftp.mkdir_p(dir_to)

    if verbose>1:
        print(f'copy starting,         time: {time.ctime()}, dir:{dir_from}')
    ssh_t.send(f"rsync -av {dir_from}/ {dir_to}/")
    test = ssh_t.expect(str_expect, verbose=1, total_timeout=timeout_cp)  ## wait for copy to end
    if test[0] is None:
        print('copy expect timed out')
    elif 'No such file or directory' in test[0]:
        print(f'copy failed: {test[0]}')
    elif test[1]==False:
        if verbose>0:
            print(f'Post copy expect timed out. time: {time.ctime()}, dir: {dir_from}')
    else:
        if verbose>1:
            print(f'copy expect completed. time: {time.ctime()}, dir: {dir_from}')

    ssh_t.receive(timeout=0.1, verbose=False, throw_error=False) ## clean any lingering receives

In [45]:
## make directory structure on MICROSCOPE
animals = ['AEG21', 'AEG22']
dates = ['2022_05_13', '2022_05_14', '2022_05_16', '2022_05_17', '2022_05_18']

In [47]:
## Define directories for where the local data is and the remote directory where the contents of that folder should be copied to (MICROSCOPE)

dir_data_from = '/n/data1/hms/neurobio/sabatini/rich/analysis/temporal_alignment'
dir_data_to = '/n/files/Neurobio/MICROSCOPE/Rich/analysis_data/face_rhythm_paper/fig_4_2pRAM'  ## contents of dir_data_local will be dumped into dir_data_MICROSCOPE


[[transfer(
    dir_from=str(Path(dir_data_from) / animal / date / 'jobNum_0'),
    dir_to=str(Path(dir_data_to) / animal / date / 'temporal_alignment'),
    sftp=sftp,
    ssh_t=ssh_t,
    str_expect='[rh183',
    verbose=2,
    timeout_cp=60*60*4,
) for date in dates] for animal in animals];

copy starting,         time: Tue Jun  7 04:46:53 2022, dir:/n/data1/hms/neurobio/sabatini/rich/analysis/temporal_alignment/AEG21/2022_05_13/jobNum_0
ent/analysis_data/face_rhythm_paper/fig_4_2pRAM/AEG21/2022_05_13/temporal_alignm 

sending incremental file list
rsync: chgrp "/n/files/Neurobio/MICROSCOPE/Rich/analysis_data/face_rhythm_paper/fig_4_2pRAM/AEG21/2022_05_13/temporal_alignment/." failed: Operation not permitted (1)
./
SxxNorm_idxS2p.npy

params.json
positionsCDR_idxS2p.npy

print_log_54442334.log

sbatch_config.sh
temporal_alignment.py
ws_signals_aligned_to_SI.h5

rsync: chgrp "/n/files/Neurobio/MICROSCOPE/Rich/analysis_data/face_rhythm_paper/fig_4_2pRAM/AEG21/2022_05_13/temporal_alignment/.SxxNorm_idxS2p.npy.VF5XUp" failed: Operation not permitted (1)

rsync: chgrp "/n/files/Neurobio/MICROSCOPE/Rich/analysis_data/face_rhythm_paper/fig_4_2pRAM/AEG21/2022_05_13/temporal_alignment/.params.json.ecvQtU" failed: Operation not permitted (1)
rsync: chgrp "/n/files/Neurobio/MICROSCOP

copy starting,         time: Tue Jun  7 04:51:17 2022, dir:/n/data1/hms/neurobio/sabatini/rich/analysis/temporal_alignment/AEG21/2022_05_18/jobNum_0
ent/analysis_data/face_rhythm_paper/fig_4_2pRAM/AEG21/2022_05_18/temporal_alignm 

sending incremental file list
rsync: chgrp "/n/files/Neurobio/MICROSCOPE/Rich/analysis_data/face_rhythm_paper/fig_4_2pRAM/AEG21/2022_05_18/temporal_alignment/." failed: Operation not permitted (1)
./
SxxNorm_idxS2p.npy

params.json

positionsCDR_idxS2p.npy

print_log_54442340.log
sbatch_config.sh

temporal_alignment.py
ws_signals_aligned_to_SI.h5

rsync: chgrp "/n/files/Neurobio/MICROSCOPE/Rich/analysis_data/face_rhythm_paper/fig_4_2pRAM/AEG21/2022_05_18/temporal_alignment/.SxxNorm_idxS2p.npy.2ey290" failed: Operation not permitted (1)

rsync: chgrp "/n/files/Neurobio/MICROSCOPE/Rich/analysis_data/face_rhythm_paper/fig_4_2pRAM/AEG21/2022_05_18/temporal_alignment/.params.json.1B0g6b" failed: Operation not permitted (1)
rsync: chgrp "/n/files/Neurobio/MICROSCO

copy starting,         time: Tue Jun  7 04:53:45 2022, dir:/n/data1/hms/neurobio/sabatini/rich/analysis/temporal_alignment/AEG22/2022_05_17/jobNum_0
ent/analysis_data/face_rhythm_paper/fig_4_2pRAM/AEG22/2022_05_17/temporal_alignm 

sending incremental file list
rsync: chgrp "/n/files/Neurobio/MICROSCOPE/Rich/analysis_data/face_rhythm_paper/fig_4_2pRAM/AEG22/2022_05_17/temporal_alignment/." failed: Operation not permitted (1)
./
SxxNorm_idxS2p.npy

params.json
positionsCDR_idxS2p.npy

print_log_54442344.log

sbatch_config.sh
temporal_alignment.py
ws_signals_aligned_to_SI.h5

rsync: chgrp "/n/files/Neurobio/MICROSCOPE/Rich/analysis_data/face_rhythm_paper/fig_4_2pRAM/AEG22/2022_05_17/temporal_alignment/.SxxNorm_idxS2p.npy.b0nHwc" failed: Operation not permitted (1)

rsync: chgrp "/n/files/Neurobio/MICROSCOPE/Rich/analysis_data/face_rhythm_paper/fig_4_2pRAM/AEG22/2022_05_17/temporal_alignment/.params.json.GOudjw" failed: Operation not permitted (1)
rsync: chgrp "/n/files/Neurobio/MICROSCOP

## Prepare username + password

In [5]:
remote_host_transfer = "transfer.rc.hms.harvard.edu"
remote_host_compute = "o2.hms.harvard.edu"
username = input('Username: ')
pw = server.pw_encode(getpass.getpass(prompt='Password: '))

Username: rh183
Password: ········


## Prepare `ssh` object

In [ ]:
## initialize ssh_transfer
ssh_t = server.ssh_interface(
    nbytes_toReceive=4096,
    recv_timeout=1,
    verbose=True,
)
ssh_t.o2_connect(
    hostname=remote_host_transfer,
    username=username,
    password=server.pw_decode(pw),
    passcode_method=1,
    verbose=1,
    skip_passcode=False,
)

In [274]:
dir_S2pOutput_remote = dir_S2pOutput_remote  ## from above
dir_s2pOutput_MICROSCOPE = (Path(dir_data_MICROSCOPE) / 'suite2p_o2_output').as_posix()  ## from above
print(f'contents of    {dir_S2pOutput_remote}    will be copied to    {dir_s2pOutput_MICROSCOPE}')

contents of    /n/data1/hms/neurobio/sabatini/rich/analysis/suite2p_output/AEG21/2022_05_14    will be copied to    /n/files/Neurobio/MICROSCOPE/Ally/Mesoscope/AEG21/2022_05_14/suite2p_o2_output


In [275]:
commands = {
    'make_dir': f"mkdir -p {dir_s2pOutput_MICROSCOPE}",
    'copy_s2p': f"cp -r {(Path(dir_S2pOutput_remote)).as_posix()}/. {dir_s2pOutput_MICROSCOPE}",
}
display(commands)

{'make_dir': 'mkdir -p /n/files/Neurobio/MICROSCOPE/Ally/Mesoscope/AEG21/2022_05_14/suite2p_o2_output',
 'copy_s2p': 'cp -r /n/data1/hms/neurobio/sabatini/rich/analysis/suite2p_output/AEG21/2022_05_14/. /n/files/Neurobio/MICROSCOPE/Ally/Mesoscope/AEG21/2022_05_14/suite2p_o2_output'}

In [276]:
ssh_t.send_receive(commands['make_dir']);

e/AEG21/2022_05_14/suite2p_o2_output



In [277]:
ssh_t.send(commands['copy_s2p'])
ssh_t.expect(str_success=f'[{username}', total_timeout=120, verbose=True);

(base) [rh183@transfer03 ~]$ 


## Clean up

In [ ]:
## delete password
del pw
gc.collect()
    
## close ssh
ssh_t.close()
del ssh_t
gc.collect()

----
# ==== Optional ====
### transfer s2p outputs from `MICROSCOPE` to local machine

In [39]:
dir_s2pOutput_MICROSCOPE = dir_s2pOutput_MICROSCOPE  ## from above
dir_s2pOutput_local = str(Path(dir_data_local).resolve() / 'suite2p_o2_output')
print(f'contents of    {dir_s2pOutput_MICROSCOPE}    will be copied to    {dir_s2pOutput_local}')

contents of    /n/files/Neurobio/MICROSCOPE/Rich/data/test/suite2p_o2_output    will be copied to    /media/rich/bigSSD/tmp_data/suite2p_o2_output


In [57]:
remote_host_transfer = "transfer.rc.hms.harvard.edu"
remote_host_compute = "o2.hms.harvard.edu"
username = input('Username: ')
pw = server.pw_encode(getpass.getpass(prompt='Password: '))

Username: rh183
Password: ········


## Prepare `sftp` and `ssh` objects

In [40]:
sftp = server.sftp_interface(hostname="transfer.rc.hms.harvard.edu", port=22)
sftp.connect(username=username, password=server.pw_decode(pw))

In [41]:
sftp.get_dir(dir_s2pOutput_MICROSCOPE, dir_s2pOutput_local)

downloading /n/files/Neurobio/MICROSCOPE/Rich/data/test/suite2p_o2_output/jobNum_0/params.json   to   /media/rich/bigSSD/tmp_data/suite2p_o2_output/jobNum_0/params.json
downloading /n/files/Neurobio/MICROSCOPE/Rich/data/test/suite2p_o2_output/jobNum_0/print_log_53419173.log   to   /media/rich/bigSSD/tmp_data/suite2p_o2_output/jobNum_0/print_log_53419173.log
downloading /n/files/Neurobio/MICROSCOPE/Rich/data/test/suite2p_o2_output/jobNum_0/batch_run_output.png   to   /media/rich/bigSSD/tmp_data/suite2p_o2_output/jobNum_0/batch_run_output.png
downloading /n/files/Neurobio/MICROSCOPE/Rich/data/test/suite2p_o2_output/jobNum_0/log.txt   to   /media/rich/bigSSD/tmp_data/suite2p_o2_output/jobNum_0/log.txt
downloading /n/files/Neurobio/MICROSCOPE/Rich/data/test/suite2p_o2_output/jobNum_0/sbatch_config.sh   to   /media/rich/bigSSD/tmp_data/suite2p_o2_output/jobNum_0/sbatch_config.sh
downloading /n/files/Neurobio/MICROSCOPE/Rich/data/test/suite2p_o2_output/jobNum_0/remote_run_s2p.py   to   /medi

## Clean up

In [43]:
## delete password
del pw
gc.collect()

## close sftp
sftp.close()
del sftp

In [42]:
ssh_t.close()
ssh_c.close()